<a href="https://colab.research.google.com/github/DanilsonCorreia/Mini-Project-Sistemas-Inteligentes/blob/main/MiniProject_SI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

# Set device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Dataset paths
train_data_path = "/content/dataset/train"
eval_data_path = "/content/dataset/Evaluation"
result_file = "result.txt"

# Data transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

eval_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Custom dataset class for evaluation data (if no subdirectories for class labels)
class CustomImageFolder(torch.utils.data.Dataset):
    def __init__(self, img_folder, transform=None):
        self.img_folder = img_folder
        self.transform = transform
        self.image_paths = [os.path.join(img_folder, img_name) for img_name in os.listdir(img_folder) if img_name.endswith(('.jpg', '.png', '.jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path  # Return image and its path for later saving predictions

# Load datasets
train_dataset = datasets.ImageFolder(train_data_path, transform=train_transforms)
eval_dataset = CustomImageFolder(eval_data_path, transform=eval_transforms)  # Using custom dataset for eval

# Split training data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

# Data loaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

# Load pre-trained model (MobileNetV2) and modify for binary classification
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 1)  # Binary classification
model = model.to(device)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_f1 = 0.0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        y_true, y_pred = [], []

        # Training step
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Collect predictions for F1 score
            preds = torch.sigmoid(outputs).cpu().detach().numpy() > 0.5
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds)

        # Calculate F1 score for training
        train_f1 = f1_score(y_true, np.array(y_pred))

        # Validation step
        model.eval()
        val_loss = 0.0
        val_y_true, val_y_pred = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Collect predictions for F1 score
                preds = torch.sigmoid(outputs).cpu().numpy() > 0.5
                val_y_true.extend(labels.cpu().numpy())
                val_y_pred.extend(preds)

        # Calculate F1 score for validation
        val_f1 = f1_score(val_y_true, np.array(val_y_pred))

        print(f"Epoch {epoch + 1} | Train Loss: {train_loss / len(train_loader):.4f}, Train F1: {train_f1:.4f}, "
              f"Val Loss: {val_loss / len(val_loader):.4f}, Val F1: {val_f1:.4f}")

        # Save the best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"Best model saved with F1: {best_f1:.4f}")

    print("Training complete!")
    return model

# Train the model
model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# Load the best model for evaluation
model.load_state_dict(torch.load("best_model.pth"))

# Generate predictions on the evaluation set
model.eval()
results = []
with torch.no_grad():
    for images, img_paths in eval_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs).cpu().numpy() > 0.5

        # Collect results: output filename and prediction
        for img_path, pred in zip(img_paths, preds):
            img_name = os.path.basename(img_path)  # Get the image filename
            pred_label = int(pred[0])  # Binary prediction (0 or 1)
            results.append(f"{img_name}\t{pred_label}")  # Tab-separated columns

# Save predictions to result.txt
with open(result_file, "w") as f:
    f.write("Filename\tPrediction\n")  # Add header row
    f.write("\n".join(results))  # Write predictions

print(f"Results saved to {result_file}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Using device: cpu


Epoch 1/10: 100%|██████████| 7/7 [00:41<00:00,  5.95s/it]


Epoch 1 | Train Loss: 0.2873, Train F1: 0.8000, Val Loss: 0.6536, Val F1: 0.8889
Best model saved with F1: 0.8889


Epoch 2/10: 100%|██████████| 7/7 [00:39<00:00,  5.60s/it]


...

Epoch 9 | Train Loss: 0.1064, Train F1: 0.9492, Val Loss: 0.3026, Val F1: 0.9091


Epoch 10/10: 100%|██████████| 7/7 [00:39<00:00,  5.68s/it]


Epoch 10 | Train Loss: 0.0317, Train F1: 0.9822, Val Loss: 0.0465, Val F1: 1.0000
Training complete!


<ipython-input-9-0d6f8de9d128>:142: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


Results saved to result.txt
